In [2]:
# We will use the official tokenization script created by the Google team
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

In [4]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [5]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [6]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [0]:
'''
from google.colab import files
import io

uploaded = files.upload()

train = pd.read_csv (io.BytesIO(uploaded['train.csv']))
test = pd.read_csv (io.BytesIO(uploaded['test.csv']))
submission = pd.read_csv(io.BytesIO(uploaded['sample_submission.csv']))
'''

In [0]:
'''
train = pd.read_csv ('train.csv')
test = pd.read_csv ('test.csv')
submission = pd.read_csv('sample_submission.csv')
'''

In [9]:
# if you are running this on Kaggle's Kernel then use this cell for data loading.

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
train = pd.read_csv ('/kaggle/input/twitter-disaster/train.csv')
test = pd.read_csv ('/kaggle/input/twitter-disaster/test.csv')
submission = pd.read_csv('/kaggle/input/twitter-disaster/sample_submission.csv')

In [23]:
from IPython.display import display

display(train.info())
display(test.info())
print("Train data shape = " , train.shape)
print("Test data shape = " , test.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
id          3263 non-null int64
keyword     3237 non-null object
location    2158 non-null object
text        3263 non-null object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


None

Train data shape =  (7613, 5)
Test data shape =  (3263, 4)


In [10]:
import tokenization

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [11]:
train_input = bert_encode(train.text.values, tokenizer, max_len=160)
test_input = bert_encode(test.text.values, tokenizer, max_len=160)
train_labels = train.target.values

In [12]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [13]:
train_history = model.fit(train_input, train_labels, validation_split=0.2, epochs=3, batch_size=16)

model.save('model.h5')

Train on 6090 samples, validate on 1523 samples
Epoch 1/3
6090/6090 [==============================] - 419s 69ms/sample - loss: 0.4812 - accuracy: 0.7778 - val_loss: 0.4023 - val_accuracy: 0.8273
Epoch 2/3
6090/6090 [==============================] - 375s 62ms/sample - loss: 0.3445 - accuracy: 0.8560 - val_loss: 0.4026 - val_accuracy: 0.8339
Epoch 3/3
6090/6090 [==============================] - 375s 62ms/sample - loss: 0.2600 - accuracy: 0.8936 - val_loss: 0.4205 - val_accuracy: 0.8359


In [19]:
# Prediction by BERT model with my tuning
model.load_weights('model.h5')
test_pred_BERT = model.predict(test_input)
test_pred_BERT_int = test_pred_BERT.round().astype('int')

In [20]:
# Prediction by BERT model with my tuning for the training data - for the Confusion Matrix
train_pred_BERT = model.predict(train_input)
train_pred_BERT_int = train_pred_BERT.round().astype('int')

In [21]:
submission.to_csv("output.csv", index=False, header=True)